### input_shape, output_shape
Dense, LSTM, Embedding layers

In [15]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution2D, Convolution1D
from keras.layers import Embedding, LSTM, SimpleRNN, TimeDistributed
from keras.layers import MaxPooling2D, MaxPooling1D

### Dense layer

In [16]:
model_1 = Sequential(
    [Dense(output_dim = 20, input_shape = (784, ))])
model_2 = Sequential(
    [Dense(output_dim = 20, input_dim = 784)])

x = np.random.uniform(size=(22, 784))
# Dense layer: 
##  input_shape  (nb_samples, input_dim)
##  output_shape (nb_samples, output_dim)
print (model_1.input_shape, model_1.output_shape)
print (model_2.input_shape, model_2.output_shape)
print (model_1.layers[0].get_output_shape_for((22, 33)))
output = model_1.predict(x)
print (output.shape)

((None, 784), (None, 20))
((None, 784), (None, 20))
(22, 20)
(22, 20)


### Embedding layer

In [17]:
word_vec_dim = 150
voca_size = 2000
n_samples = 100
seq_length = 3
model_1 = Sequential([Embedding(output_dim = word_vec_dim, input_dim = voca_size)])
# Embedding layer:
## input_shape  (nb_samples, sequence_length)
## output_shape (nb_samples, sequence_length, output_dim)
print (model_1.layers[0].input_shape, model_1.layers[0].output_shape)
x = np.random.randint(low=0, high=2000, size=(n_samples, seq_length))
output = model_1.predict(x)
print (output.shape) # (n_samples, seq_length, word_vec_dim)

((None, None), (None, None, 150))
(100, 3, 150)


### SimpleRNN

In [18]:
input_dim = 50
output_dim = 150
seq_length = 10
model = Sequential()
#model.add(SimpleRNN(output_dim=output_dim, input_dim=input_dim, input_length=seq_length, return_sequences=True))
model.add(SimpleRNN(output_dim=output_dim, input_shape=(seq_length, input_dim), return_sequences=True))
# SimpleRNN
## input_shape  (nb_samples, timesteps, input_dim)
## output_shape (nb_samples, timesteps, output_dim)
print (model.input_shape, model.output_shape)

((None, 10, 50), (None, 10, 150))


### LSTM layer

In [19]:
input_dim = 150
hidden_dim = 300
seq_length = 10
n_samples = 20

model_1 = Sequential([LSTM(output_dim = hidden_dim, input_dim = input_dim, input_length = seq_length)])
# LSTM layer
## input_shape  (nb_samples, timesteps, input_dim)
## output_shape: 
####          return_sequences==True: (nb_samples, timesteps, input_dim) 
####          return_sequences==False: (nb_samples, input_dim) ### only the last output returned
print (model_1.layers[0].input_shape)
print (model_1.layers[0].output_shape)
x = np.random.uniform(size=(n_samples, seq_length, input_dim))
output = model_1.predict(x)
print (output.shape)

model_2 = Sequential([LSTM(output_dim = hidden_dim, input_dim = input_dim, input_length = seq_length, 
                           return_sequences=True)]) ## return sequences
output = model_2.predict(x)
print (output.shape)

(None, 10, 150)
(None, 300)
(20, 300)
(20, 10, 300)


### wrapper: TimeDistributed

In [29]:
voca_size = 1000
seq_length = 10
word_vec_dim = 100
hidden_dim = 300
output_dim = 200
model = Sequential()
model.add(Embedding(output_dim=word_vec_dim, input_dim=voca_size, input_length=seq_length))
model.add(SimpleRNN(output_dim=hidden_dim, activation='sigmoid', return_sequences=True))
model.add(TimeDistributed(Dense(output_dim=output_dim, activation='softmax')))
print (model.input_shape, model.output_shape)

((None, 10), (None, 10, 200))


### Convolution2D

In [67]:
nb_filter = 64
rf_size = (5, 5) # receptive field size
input_shape = (3, 256, 256) # 256x256 RGB picture
strides = (2, 2)
model = Sequential()
model.add(Convolution2D(nb_filter=nb_filter, nb_row=rf_size[0], nb_col=rf_size[1], input_shape=input_shape, 
                                 border_mode='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# Convolution2D layer
## dim_ordering == 'th'
####   input_shape  (nb_samples, channels, rows, cols)
####   output_shape (nb_samples, nb_filter, new_rows, new_cols)

## dim_ordering == 'tf'
####   input_shape  (nb_samples, rows, cols, channels)
####   output_shape (nb_samples, new_rows, new_cols, nb_filter)

print ('input_shape: {0}'.format(model.layers[0].input_shape))
x = np.random.uniform(size=(1, 3, 256, 256))

output = model.predict(x)
print ('output_shape: {0}'.format(output.shape))
ws = model.layers[0].get_weights()
print ('conv layer: weight_shape: {0}, bias_shape: {1}'.format(ws[0].shape, ws[1].shape)) # kernel shape (nb_filters, nb)


input_shape: (None, 3, 256, 256)
output_shape: (1, 64, 128, 128)
conv layer: weight_shape: (64, 3, 5, 5), bias_shape: (64,)


### Convolution1D

In [38]:
nb_filter = 64
rf_size = 3
input_dim = 32 ## channels to Convolution2D
model = Sequential([Convolution1D(nb_filter=nb_filter, filter_length=rf_size, input_dim=32)])
# Convolution1D
## input_shape  (nb_samples, timesteps, channels)
## output_shape (nb_samples, new_timesteps, nb_filter)
print ('input_shape: {0}'.format(model.input_shape))
x = np.random.uniform(size=(1, 10, input_dim))
output = model.predict(x)
print ('output_shape: {0}'.format(output.shape))
ws = model.layers[0].get_weights()
print ('weight_shape: {0}, bias_shape: {1}'.format(ws[0].shape, ws[1].shape))

input_shape: (None, None, 32)
output_shape: (1, 8, 64)
weight_shape: (64, 32, 3, 1), bias_shape: (64,)
